

```
# This is formatted as code
```

### **Actualización 1.8:** (nACHO 14/12 a las 9:53)

---


* Cambio **IMPORTANTE** en archivos:
  - Se importa TEMP_SCHEMES.py para los esquemas temporales, que solo dependen de U, dt, t, F. El esquema embebido necesita de ERK_AUX.py
  - Se importa EDO.py para CauchyProblem que solo depende de (U,t,Scheme,F), es decir, sin modificar para alojar Nb y Nc.
  - Se modifica F_NBody, que no lleva implementada Nc como argumento
  - Para evitar dichos cambios, se wrappea la F en el main
  - Creación del bucle para pintar en fijo las órbitas
  - Actualización del 3d con los nuevos r_rec que hace el reshape de la r obtenida, asi como la u con U_rec






**ARCHIVOS EN USO**
  

*   TEMP_SCHEMES.py
*   ERK_AUX.py
*   fNbody.py
*   initialConditions.py
*   EDO.py
*   main.py si quiero correr en local. MODIFICADO A LA v1.7

**ARCHIVOS COPIADOS SIN USO**
*   Cauchy_Problem.py antiguo de Cauchy modificado de Javi
*   NBODY.py el de Sergio de n Body sin paralelizar
*   Physics.py para pruebas con Kepler

**COMENTARIOS**
- Nacho está implementando masas y esferas de influencia. El punto de partida debería ser esta versión.
- Tarda más con GPU que sin ella en el collab para 10 bodies y 100 secs. Habrá que definir un delta de t que sea de 0.01 a 0.001
- Estudio comparativo en tiempos para un caso paralelizado y otro sin?

# Import Google Drive

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

print('Enter the path:')
path = input()
import sys
sys.path.insert(0,'/content/drive/MyDrive/'+path)


# Upload modules manually

In [ ]:
from google.colab import files
files.upload()

# Import functions

In [ ]:
import numpy as np
import math
from numba import cuda
from scipy.optimize import newton
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

from EDO import CauchyProblem # Cauchy(F,t,U0,Scheme)
from TEMP_SCHEMES import Euler_Scheme, CrankNicolson_Scheme, RK4_Scheme, InvEuler_Scheme, LeapFrog, Embedded_RK # Scheme(U,dt,t,F)
from fNbody import F_NBody # (U,t,Nb,Nc,M)
from initialConditions import initial_cond, solarsystem
from Physics import Kepler

# Información de la GPU

In [ ]:
from numba import cuda
gpu = cuda.get_current_device()
print("name = %s" % gpu.name)
print("maxThreadsPerBlock = %s" % str(gpu.MAX_THREADS_PER_BLOCK))
print("maxBlockDimX = %s" % str(gpu.MAX_BLOCK_DIM_X))
print("maxBlockDimY = %s" % str(gpu.MAX_BLOCK_DIM_Y))
print("maxBlockDimZ = %s" % str(gpu.MAX_BLOCK_DIM_Z))
print("maxGridDimX = %s" % str(gpu.MAX_GRID_DIM_X))
print("maxGridDimY = %s" % str(gpu.MAX_GRID_DIM_Y))
print("maxGridDimZ = %s" % str(gpu.MAX_GRID_DIM_Z))
print("maxSharedMemoryPerBlock = %s" % str(gpu.MAX_SHARED_MEMORY_PER_BLOCK))
print("asyncEngineCount = %s" % str(gpu.ASYNC_ENGINE_COUNT))
print("canMapHostMemory = %s" % str(gpu.CAN_MAP_HOST_MEMORY))
print("multiProcessorCount = %s" % str(gpu.MULTIPROCESSOR_COUNT))
print("maxThreadsPerMultiprocessor = %s" % str(gpu.MAX_THREADS_PER_MULTI_PROCESSOR))
print("warpSize = %s" % str(gpu.WARP_SIZE))
print("unifiedAddressing = %s" % str(gpu.UNIFIED_ADDRESSING))
print("pciBusID = %s" % str(gpu.PCI_BUS_ID))
print("pciDeviceID = %s" % str(gpu.PCI_DEVICE_ID))

cuda.detect()

# main

In [ ]:
print('Enter type of simulation: "Solar System" (S)  or "Random" (R)')
sim  = str(input())

if sim == "Solar System" or sim == "S" or sim == "s":
  Nb = 11
  Nc = 3

  U0, M = solarsystem()

  print("Upload solar system data")

else:

  print('Enter number of bodies:')
  Nb = int(input()) # 1024*72*2 # nº cuerpos tiene que ser 1024 (nº threads por bloque) * 72 (nº bloques o SM) * A (nº entero a elegir)
  Nc = 3 # nº coord

  U0, M = initial_cond(Nb,Nc) #ok

t0 = 0 # initial time
print('Enter duration of simulation [s]:')
tf = int(input()) # final time
dt = np.amax([0.1,0.0001*tf]) # time step
N =  int(np.floor(tf/dt)+1) # number of time steps
t = np.linspace(t0, tf, N)

U0, M = initial_cond(Nb,Nc) #ok

def F(U,t): # wrapp de la F para no cambiar Cauchy ni los esquemas
  return F_NBody(U, t, Nb, Nc, M)

U = CauchyProblem(F, t, U0, Embedded_RK)
U_rec = np.reshape(U,(N, Nb, 2, Nc))
r_rec = np.reshape(U_rec[:,:,0,:],(N,Nb,Nc))

for i in range(0,Nb):
  np.savetxt('/content/drive/MyDrive/dataU_N_' + str(Nb) + '_0_Nc.txt', r_rec[i,:,:])
        
print('Finished!')
        
 





# 3D plot animation

In [ ]:
# 3D PLOT ANITAMED
def update_lines(num, walks, lines):
    for line, walk in zip(lines, walks):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(walk[:num, :2].T)
        line.set_3d_properties(walk[:num, 2])
    return lines

def paint(N,Nc,r,index):
  line=np.zeros((N,Nc))
  for i in range(N):
    line[i,:]=r[i,index,:]
  return line

# Redimensionamiento para el plot
k = np.amax(np.linalg.norm(r_rec[1,:,:]))
print(np.linalg.norm(r_rec[1,:,:]))
print(k)
r_rec = r_rec/(0.1*k)
print(r_rec)

# Data: Nb lines as (num_steps, 3) arrays
datas = [paint(N,Nc,r_rec,index) for index in range(Nb)]
#print ("datas: ",datas)

# Attaching 3D axis to the figure
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Create lines initially without data
lines = [ax.plot([], [], [])[0] for _ in datas]

# Setting the axes properties
ax.set(xlim3d=(-10, 10), xlabel='X')
ax.set(ylim3d=(-10, 10), ylabel='Y')
ax.set(zlim3d=(-10, 10), zlabel='Z')

# Creating the Animation object
line_ani = animation.FuncAnimation(fig, update_lines, N//2, fargs=(datas, lines), interval=2500)
f = r"/content/drive/MyDrive/animation.mp4" 
#f = r"/content/drive/MyDrive/" + path + "/animation.mp4" 
writervideo = animation.FFMpegWriter(fps=60) 
line_ani.save(f, writer=writervideo, dpi=300)
#plt.show()

In [ ]:
# 3D PLOT FIJO
colors = ['b','r','g','m','y','c']
fig = plt.figure(2)
ax = fig.add_subplot(projection='3d')
for i in range(Nb):
  ax.plot(r_rec[:, i, 0], r_rec[:, i, 1], r_rec[:, i, 2])#, colors[i])

plt.title(f'N = {Nb} body problem: 3D projection')
plt.xlabel("x")
plt.ylabel("y")

#plt.grid(True)
plt.show()